In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

1. Who are the top 5 users in Bangalore with the highest number of followers? List their login in order, comma-separated.

In [4]:
df = pd.read_csv('/content/users.csv')
df_r = pd.read_csv('/content/repositories.csv')
df.head(5)

,login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at,surname,bio_len,bio_word_count
0,krishnaik06,Krish C Naik,PANASONIC IIC,Bangalore,krishnaik06@gmail.com,False,Data Scientist with ML and Deep Learning expe...,329,30869,0,2016-06-20T09:25:45Z,Naik,52.0,8
1,championswimmer,Arnav Gupta,JIOCINEMA,"Bangalore, India",dev@championswimmer.in,True,Director of Engineering - @JioCinema ┃\r\nPast...,351,4661,29,2012-01-13T07:41:40Z,Gupta,150.0,21
2,arpitbbhayani,Arpit Bhayani,DICEDB,Bangalore,NaN,True,"Creator of @DiceDB • ex-Google Dataproc, ex-Am...",188,4549,3,2013-06-20T06:40:12Z,Bhayani,60.0,9
3,manjunath5496,Manjunath.R,MYW3SCHOOLS.COM,"Bangalore, India",manjunath5496@gmail.com,False,"""Science is not only a disciple of reason but,...",1563,4081,9313,2016-06-16T05:02:32Z,Manjunath.R,101.0,18
4,tanaypratap,Tanay Pratap,INVACT,"Bangalore, India",tanay.mit@gmail.com,False,Founder & CEO @invact . Teacher @neogcamp,40,3030,1,2014-12-17T05:15:06Z,Pratap,42.0,7


2. Who are the 5 earliest registered GitHub users in Bangalore? List their login in ascending order of created_at, comma-separated.

In [5]:
df_bangalore = df[df['location'].str.contains("Bangalore", case=False, na=False)]
df_sorted = df_bangalore.sort_values(by='created_at').head(5)

earliest_users = ','.join(df_sorted['login'].tolist())
print(earliest_users)

anandology,irfn,jace,abhisek,abhin4v


3. What are the 3 most popular license among these users? Ignore missing licenses. List the license_name in order, comma-separated.

In [6]:
df_licenses = df_r.dropna(subset=['license_name'])

top_licenses = df_licenses['license_name'].value_counts().head(3).index.tolist()

top_licenses_str = ','.join(top_licenses)
print(top_licenses_str)

mit,apache-2.0,other


4. Which company do the majority of these developers work at?

In [7]:
company_counts = df['company'].value_counts()
most_common_company = company_counts.idxmax()
most_common_count = company_counts.max()

print(f"The majority of developers work at: {most_common_company} with {most_common_count} developers.")

The majority of developers work at: GOOGLE with 14 developers.


Which programming language is most popular among these users?

In [8]:
language_counts = df_r['language'].value_counts()
most_popular_language = language_counts.idxmax()
most_popular_count = language_counts.max()

print(f"The most popular programming language among these users is: {most_popular_language} with {most_popular_count} repositories.")

The most popular programming language among these users is: JavaScript with 9355 repositories.


6. Which programming language is the second most popular among users who joined after 2020?

In [9]:
df['created_at'] = pd.to_datetime(df['created_at'])
filtered_users = df[df['created_at'] > '2020-01-01']
filtered_repos = df_r[df_r['login'].isin(filtered_users['login'])]
language_counts = filtered_repos['language'].value_counts()
second_most_popular_language = language_counts.index[1]
second_most_popular_count = language_counts.iloc[1]

print(f"The second most popular programming language among users who joined after 2020 is: {second_most_popular_language} with {second_most_popular_count} repositories.")

The second most popular programming language among users who joined after 2020 is: HTML with 652 repositories.


7. Which language has the highest average number of stars per repository?

In [10]:
average_stars = df_r.groupby('language')['stargazers_count'].mean()
highest_avg_stars_language = average_stars.idxmax()
highest_avg_stars_value = average_stars.max()

print(f"The programming language with the highest average number of stars per repository is: {highest_avg_stars_language} with an average of {highest_avg_stars_value:.2f} stars.")

The programming language with the highest average number of stars per repository is: Pascal with an average of 92.00 stars.


8. Let's define leader_strength as followers / (1 + following). Who are the top 5 in terms of leader_strength? List their login in order, comma-separated.

In [11]:
df['leader_strength'] = df['followers'] / (1 + df['following'])
top_leaders = df.nlargest(5, 'leader_strength')['login']
top_leaders_list = ','.join(top_leaders)
top_leaders_list

'krishnaik06,Hack-with-Github,laxmimerit,tanaypratap,abhishh1'

9. What is the correlation between the number of followers and the number of public repositories among users in Bangalore?

In [13]:
correlation = df['followers'].corr(df['public_repos'])
correlation_rounded = round(correlation, 3)
correlation_rounded

0.192

10. Does creating more repos help users get more followers? Using regression, estimate how many additional followers a user gets per additional public repository.

In [16]:
from sklearn.linear_model import LinearRegression

X = df[['public_repos']]
y = df['followers']

model = LinearRegression().fit(X, y)
additional_followers_per_repo = model.coef_[0]
rounded = round(additional_followers_per_repo, 3)
rounded

2.357

11. Do people typically enable projects and wikis together? What is the correlation between a repo having projects enabled and having wiki enabled?

In [17]:
correlation = df_r['has_projects'].corr(df_r['has_wiki'])
df_r['has_projects'].value_counts() ,df_r['has_wiki'].value_counts()

correlation_rounded = round(correlation, 3)
correlation_rounded

0.198

12. Do hireable users follow more people than those who are not hireable?

In [19]:
hierable = df[df['hireable'] == True]
non_hierable = df[df['hireable'] != True]


difference = hierable['following'].mean()-non_hierable['following'].mean()
difference_rounded = round(difference, 3)


print(difference_rounded)

79.91


13. Some developers write long bios. Does that help them get more followers? What's the impact of the length of their bio (in Unicode words, split by whitespace) with followers? (Ignore people without bios)

In [21]:
from sklearn.linear_model import LinearRegression


df_with_bio = df
df_with_bio['bio'] = df_with_bio['bio'].fillna('')
df_with_bio = df_with_bio[df_with_bio['bio'].notna() & (df_with_bio['bio'] != '')].copy()
df_with_bio.loc[:, 'bio_len'] = df_with_bio['bio'].str.len()
df_with_bio['bio_word_count'] = df_with_bio['bio'].str.split().apply(len)

X = df_with_bio[['bio_word_count']].values.reshape(-1,1)
y = df_with_bio['followers']

model = LinearRegression()
model.fit(X, y)
slop = model.coef_[0]
rounded = round(slop, 3)
rounded

0.339

14. Who created the most repositories on weekends (UTC)? List the top 5 users' login in order, comma-separated

In [22]:
df_r['created_at'] = pd.to_datetime(df_r['created_at'], utc=True)
weekend_repos = df_r[df_r['created_at'].dt.weekday.isin([5, 6])]
weekend_repo_counts = weekend_repos['login'].value_counts().head(6)

top_5_users = ','.join(weekend_repo_counts.index)

print(top_5_users,weekend_repo_counts)

manjunath5496,itaditya,avinassh,narayanr7,Tivotal,sangam14 login
manjunath5496    405
itaditya         131
avinassh         127
narayanr7        126
Tivotal          125
sangam14         125
Name: count, dtype: int64


15. Do people who are hireable share their email addresses more often?

In [24]:
hireable = df[df['hireable'] == True]
non_hireable = df[df['hireable'] != True]

hireable_email_fraction = hireable['email'].notna().mean()
non_hireable_email_fraction = non_hireable['email'].notna().mean()

email_fraction_difference = round(hireable_email_fraction - non_hireable_email_fraction, 3)

print(email_fraction_difference)

0.18


16. Let's assume that the last word in a user's name is their surname (ignore missing names, trim and split by whitespace.) What's the most common surname? (If there's a tie, list them all, comma-separated, alphabetically)

In [25]:
df['surname'] = df['name'].apply(lambda x: x.split()[-1])
surname_counts = df['surname'].value_counts()
max_count = surname_counts.max()
most_common_surnames = ','.join(sorted(surname_counts[surname_counts == max_count].index))

print(f"Most common surname(s): {most_common_surnames}")
print(f"Number of users with the most common surname: {max_count}")

Most common surname(s): Kumar,Singh
Number of users with the most common surname: 21
